In [ ]:
from time import time  # Used to show loading and execution time,
t0 = time()

# IPOL with a notebook
## Example with the simplest colour balance algorithm

## Notebook-related imports

Only `ipywidgets` is strictly necessary

In [ ]:
# the following two are helpful to automatically load example images from a folder, but are not strictly needed
from pathlib import Path

pathlib imported

In [ ]:
from glob import glob

glob imported

In [ ]:
# PIL can crop images and rewrites them as byte arrays with io
import io

io imported

In [ ]:
from PIL import Image

pil

In [ ]:
import ipywidgets as widgets  # to select inputs

ipywidgets

In [ ]:
import imageio  # ipywidgets reads images as a byte array, imageio can then convert it to a more useful numpy array

imageio

In [1]:
from matplotlib import pyplot as plt  # Will be used to plot results

In [1]:
matplotlib

Using matplotlib backend: Qt5Agg


**The following line can make the resulting plots be seen interactively. `ipympl` is required.**

This also means a few seconds are needed just to show the results. It can be disabled, and outputs will be shown statically.

In [5]:
%matplotlib widget

## Imports related to the code.

In [2]:
import numpy as np  # used in the code just below

from simplest_color_balance import scb  # our code

### Demo-specific code

For instance : code to display results, or functions to run the code that don't fit in just a few lines but should not be in a separate file because they are specific to the notebook.

Here we write a function to plot the image histogram.

In [19]:
def histogram(img, ax):
    if img.ndim == 3:
        sz = img[:, :, 0].size
        hist_r, _ = np.histogram(img[:, :, 0].ravel(),256,[0,256])
        hist_g, _ = np.histogram(img[:, :, 1].ravel(),256,[0,256])
        hist_b, _ = np.histogram(img[:, :, 2].ravel(),256,[0,256])
        hist_r = hist_r / sz
        hist_g = hist_g / sz
        hist_b = hist_b / sz
        ax.plot(hist_r, color='red', lw=.8)
        ax.plot(hist_g, color='green', lw=.8)
        ax.plot(hist_b, color='blue', lw=.8)
    elif img.ndim == 1:
        hist, _ = np.histogram(img.ravel(), 256, [0, 256], lw=.8)
        ax.plot(hist)
    else:
        raise ValueError(img.ndim)


## Now we define the widgets

### Image selection

We can upload an image

In [ ]:
w_up_img = widgets.FileUpload(
    accept='image/*',
    description="Input image"
)

Or we can choose one for examples, this takes a few more lines:

First we gather the example images

In [ ]:
images = sorted(list(map(Path, glob('images/original/*'))))[:10]

We create a list of widgets, each widget shows an images

In [ ]:
ws_images = [widgets.Image(width=150, height=150, value=open(img, 'rb').read()) for img in images]

Alongside the list of image widgets, we create a list of buttons to select each image. Here, the button description is the name of the image.

In [ ]:
images_labels_text = [img.stem for img in images]

ws_buttons = [widgets.Button(description=label, button_style='') for label in images_labels_text]

Merge the "image" and "button" widgets: each widget in this list shows the image, and below a button to "select" it.

In [ ]:
ws_stack_image_button = [widgets.VBox([i, b]) for i, b in zip(ws_images, ws_buttons)]

Turn the list of widgets into one widget, which shows the different images in a grid layout.

In [ ]:
w_img_choice = widgets.GridBox(ws_stack_image_button, layout=widgets.Layout(grid_template_columns="repeat(5, 200px)"))

`w_selected_img` is used to show the currently selected image, be it uploaded or selected among the examples.

In [ ]:
w_selected_img = widgets.Image(width=300, height=300)

### Image Cropping

Now we show a checkbox to select whether to crop the image.

In [ ]:
w_choose_crop = widgets.Checkbox(value=False, description="Crop image?")

And two sliders to select by how much.

In [ ]:
w_crop_slider_x = widgets.IntRangeSlider(
    min=0,
    max=0,
    value=(0, 0),
    step=1,
    description='X Crop')

w_crop_slider_y = widgets.IntRangeSlider(
    min=0,
    max=0,
    value=(0, 0),
    step=1,
    description='Y Crop')

By default, the two sliders are hidden : they will only be shown if the checkbox to crop is clicked.

In [ ]:
w_crop_slider_x.layout.visibility = 'hidden'
w_crop_slider_y.layout.visibility = 'hidden'

### Parametres of the simplest color balance

The SCB algorithms takes two parametres : the percentage of pixels to saturate in both ends of the spectrum.

They can be set in two ways : using a slider, or by writing them down in a textbox. We thus create one widget for the slider, and one for each textbox.

In [ ]:
w_s = widgets.FloatRangeSlider(
    min=0,
    max=100,
    value=(1.5, 98.5),
    step=.1,
    readout_format='.1f',
    description='Percentage of pixels to keep')
w_s0 = widgets.FloatText(description='s0', value=w_s.value[0])
w_s1 = widgets.FloatText(description='s1', value=w_s.value[1])

### Button to run the code once parametres are set

In [ ]:
w_run = widgets.ToggleButton(value=False, description='run', icon='cog')

## Observation

Now that the widgets are defined, we create observation functions to interact with them:

First, the generic function that can update `w_selected_img` with a new image. It also sets the range of the sliders to crop the image:

In [ ]:
def set_new_image(new_img):
    w_selected_img.value = new_img
    w_selected_img.stored = w_selected_img.value
    Y, X = imageio.imread(w_selected_img.value).shape[:2]
    w_crop_slider_x.max = X
    w_crop_slider_y.max = Y
    w_crop_slider_x.value = (0, X)
    w_crop_slider_y.value = (0, Y)
    w_choose_crop.value = False

Observation functions for image upload (`observe_w_up_img`) or image selection (`observe_w_img_choice`). As buttons give no easy way of telling which button was pressed, we simply add a new property to them (`id`)

In [ ]:
def observe_w_up_img(change):
    set_new_image(change.new[0])

def observe_w_img_choice(b):
    set_new_image(ws_images[b.id].value)

Link the observation functions to the widgets.

In [ ]:
for i in range(len(ws_buttons)):
    ws_buttons[i].id = i
    ws_buttons[i].on_click(observe_w_img_choice)

w_up_img.observe(observe_w_up_img, names='data')

Load the first example image as a default choice.

In [ ]:
observe_w_img_choice(ws_buttons[0])

Now for cropping:
* `observe_w_choose_crop`: When the checkbox is clicked, make visible the crop sliders. When it is unclicked, hide them and restore the image to full size
* `observe_w_crop_sliders`: When the crop sliders are moved, 
* If it is selected in text, update sliders value, and vice versa


In [ ]:
def observe_w_choose_crop(change):
    if change.new:
        w_crop_slider_x.layout.visibility = 'visible'
        w_crop_slider_y.layout.visibility = 'visible'
    else:
        w_crop_slider_x.layout.visibility = 'hidden'
        w_crop_slider_y.layout.visibility = 'hidden'
        w_selected_img.value = w_selected_img.stored
    

def observe_w_crop_sliders(_):
    left, right = w_crop_slider_x.value
    top, bottom = w_crop_slider_y.value
    img = Image.open(io.BytesIO(w_selected_img.stored))
    img = img.crop((left, top, right, bottom))
    out = io.BytesIO()
    img.save(out, format='png')
    out.seek(0)
    w_selected_img.value = out.read()
    

w_choose_crop.observe(observe_w_choose_crop, 'value')
w_crop_slider_x.observe(observe_w_crop_sliders, 'value')
w_crop_slider_y.observe(observe_w_crop_sliders, 'value') 
    

saturation parametres:
* `observe_w_s0`, `observe_w_s1` : if value is selected in textbox, force it within the accepted range, and update the slider
* `observe_w_s` : if value is selected with the slider, update the textboxes

In [ ]:
def observe_w_s0(change):
    old_s0, s1 = w_s.value
    new_s0 = min(s1, max(0, change.new))
    w_s0.value = new_s0
    w_s.value = new_s0, s1
    
def observe_w_s1(change):
    s0, old_s1 = w_s.value
    new_s1 = min(100, max(s0, change.new))
    w_s1.value = new_s1
    w_s.value = s0, new_s1
    
def observe_w_s(change):
    s0, s1 = change.new
    w_s0.value = s0
    w_s1.value = s1
    

# load observations


w_s0.observe(observe_w_s0, names='value')
w_s1.observe(observe_w_s1, names='value')
w_s.observe(observe_w_s, names='value')

`launch` is to be called every time the input changes. We only want the code to run once everything is selected, so if the run button isn't on we return immediately. If it is on, we disable it and run the code.

This function processes the inputs (for instance, here convert the input image from byte string to numpy array), runs the code, displays the results, and registers them to the IPOL archive.

In [ ]:
def launch(ready, img, s0, s1):
    if not ready:
        return
    t0 = time()
    w_run.value = False
    img = imageio.imread(img)
    t1 = time()
    out = run(img, s0, s1)
    t2 = time()
    display_results(img, s0, s1, out)
    register_results(img, s0, s1, out) 
    t3 = time()
    print(f"""
    Data conversion time: {t1-t0:.3f}s.
    Code execution time: {t2-t1:.3f}s.
    Results display time: {t3-t1:.3f}s.
    Total execution time: {t3-t0:.3f}s.
    """
    )

`run` is the function that takes input as processed by `launch`, runs the full code, and returns the output.

In [ ]:
def run(img, s0, s1):
    s0 = s0/100
    s1 = 1 - s1/100    
    out = scb(img, s0, s1)
    return out

`display_results` takes the inputs and outputs and displays them in the notebook.

In [ ]:
def display_results(img, s0, s1, out):
    fig, ax = plt.subplots(2, 2, figsize=(10, 5))
    ax[0, 0].imshow(img)
    ax[0, 0].set_title('Input')
    ax[0, 0].axis('off')
    ax[0, 1].imshow(out)
    ax[0, 1].set_title('Output')
    ax[0, 1].axis('off')
    histogram(img, ax[1, 0])
    ax[1, 0].set_title('Input histogram')
    histogram(out*255, ax[1, 1])
    ax[1, 1].set_title('Output histogram')
    plt.show()

Finally, `register_results` is used to archive the experiment to IPOL.

In [ ]:
def register_results(*args, **kwargs):
    return

`w_out` is used to call the `launch` function whenever the input changes (`launch` will then return immediately if the run button wasn't last pressed).

There are options to directly add a run button and only run the `launch` function then, but it does not seem compatible with complex inputs such as the one we need.

In [ ]:
w_out = widgets.interactive_output(launch, {'ready': w_run, 'img': w_selected_img, 's0': w_s0, 's1': w_s1})

Create a VBox to display all widgets together (TBD: it's possible to create a better layout)

In [ ]:
w_all_widgets = widgets.VBox(
    [
        w_up_img,  # upload an image
        w_img_choice,  # choose an image in selection
        w_selected_img,  # shows the selected/uploaded image
        widgets.HBox([w_choose_crop, w_crop_slider_x, w_crop_slider_y]),  # crop the image
        widgets.HBox([w_s0, w_s, w_s1]),  # Simplest Colour Balance saturation parametres
        w_run,  # Run button
        w_out,  # Interactive output
    ],
    align_items='center'
)

#w_run.on_click(callback)

# display
#display(vb)

In [ ]:
display(w_all_widgets)
t1 = time()
print(f"Notebook loaded in {t1-t0:.3f}s.")